In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim




In [ ]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

# Load the dataset
trainset = torchvision.datasets.ImageFolder(root='path_to_train_directory', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# Define a simple CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the network
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [2]:
# Install dependencies as necessary
# %pip install -qr requirements.txt  # Install dependencies (ignore errors)

import torch
from IPython.display import Image, clear_output  # To display images
import yaml
import time

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Define number of classes based on YAML
with open(r"D:\asl\yolov5\Sign_language_data\data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])


Using device: cpu


In [3]:
import torch
print(torch.cuda.is_available())


False


In [4]:

# Define the content of custom_yolov5s.yaml
custom_yaml_content = """
# Parameters
nc: {num_classes}  # Number of classes
depth_multiple: 0.33  # Model depth multiple
width_multiple: 0.50  # Layer channel multiple

# Anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # Cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # Cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # Cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # Cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]
"""


In [5]:

# Define the file path for custom_yolov5s.yaml
custom_yaml_path = r"D:\asl\yolov5\models\custom_yolov5s.yaml"

# Write the content to custom_yolov5s.yaml
with open(custom_yaml_path, 'w') as f:
    f.write(custom_yaml_content)


In [6]:

# Define training parameters and start training
start_time = time.time()

%cd D:\asl\yolov5
!python train.py --img 416 --batch 16 --epochs 10 --data "Sign_language_data\data.yaml" --cfg ".\models\custom_yolov5s.yaml" --weights "yolov5s.pt" --name yolov5s_results --cache

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")


D:\asl\yolov5
^C
Execution time: 17.309468984603882 seconds


In [1]:
import time
from tqdm import tqdm  # Import tqdm for progress bars

start_time = time.time()

%cd D:\asl\yolov5

# Define training parameters
epochs = 10

# Start training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    
    # Training phase
    print("Training...")
    for i, batch in enumerate(tqdm(train_loader)):
        # Your training code here
        pass
    
    # Validation phase
    print("Validation...")
    for i, batch in enumerate(tqdm(val_loader)):
        # Your validation code here
        pass

end_time = time.time()
execution_time = end_time - start_time
print("Total execution time:", execution_time, "seconds")


D:\asl\yolov5
Epoch 1/10
Training...


NameError: name 'train_loader' is not defined

In [8]:
import time
import subprocess

start_time = time.time()

# Define the command to execute
command = [
    'python', 'train.py',
    '--img', '416',
    '--batch', '16',
    '--epochs', '10',
    '--data', 'Sign_language_data\data.yaml',
    '--cfg', '.\models\custom_yolov5s.yaml',
    '--weights', 'yolov5s.pt',
    '--name', 'yolov5s_results',
    '--cache'
]

# Execute the command and capture the output
try:
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=r'D:\asl\yolov5', check=True)
    print(result.stdout.decode('utf-8'))
except subprocess.CalledProcessError as e:
    print("An error occurred:", e)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")
